Homework 4 - Multi-Class Classification 

Nishchay Patel

1. Importing the libraries needed and reading the Dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


In [ ]:
data = pd.read_csv('performance.csv')
data.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
print(data.sample(5))

In [ ]:
data.isnull().sum()

In [ ]:
print(data.nunique())

In [ ]:

getting_count = data["Target"]

label_counts = getting_count.value_counts()
print("Label Distribution:")
print(label_counts)

plt.figure(figsize=(10, 6))
sns.barplot(x=label_counts.index, y=label_counts.values)
plt.title('Label Distribution')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:

# Step 1: Make sure that your column names are correct
data.columns = data.columns.str.strip()  # Remove any leading/trailing spaces
data.columns = data.columns.str.replace(' ', '_')  # Replace spaces with underscores

# Step 2: Encode the target variable using LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Target'])  # Target variable encoded

# Step 3: Select the feature columns (X) by dropping the target column
X = data.drop('Target', axis=1)

# Step 4: Define the categorical columns (nominal and ordinal)
categorical_columns_nominal = ['Gender', 'Merit_Scholarship', 'Uni_Transport', 'Learn_Mode', 'Smartphone', 'PC', 'Program', 'Skills1', 'Interest_Area1']
categorical_columns_ordinal = ['Eng_Proficiency', 'Relationship']  # Modify if needed based on the unique counts

# Step 5: Define the numerical columns
numerical_columns = ['Age', 'Adm_Year', 'HSC_Pass_Year', 'Curr_Sem1', 'Study_Hours1', 
                     'Study_Sessions1', 'SM_Hours1', 'Avg_Attendance1', 'Skill_Dev_Hours1', 
                     'Prev_SGPA1', 'Curr_CGPA1', 'Completed_Credits1', 'Family_Income1']

# Step 6: Create a ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        # OneHotEncoder for nominal categorical columns
        ('cat_nominal', OneHotEncoder(drop='first'), categorical_columns_nominal),
        
        # StandardScaler for numerical columns
        ('num', StandardScaler(), numerical_columns)
    ])

# Step 7: Apply ColumnTransformer to the feature set (X) to preprocess the data
X_processed = preprocessor.fit_transform(X)

# Step 8: Split the dataset into training (70%) and testing (30%) sets using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# Now, X_train, X_test, y_train, and y_test are ready for use in machine learning tasks.


In [ ]:
X_train_dense = X_train.toarray()

# Convert to DataFrame for sampling (optional)
X_train_df = pd.DataFrame(X_train_dense)

# Now you can sample
X_train_df.sample(3)

In [ ]:
X_test_dense = X_test.toarray()

# Convert to DataFrame for sampling (optional)
X_test_df = pd.DataFrame(X_test_dense)

# Now you can sample
X_test_df.sample(3)

In [ ]:
y_train_df = pd.DataFrame(y_train, columns=['Target'])
y_test_df = pd.DataFrame(y_test, columns=['Target'])

In [ ]:
print(X_train_df.shape,y_train_df.shape)
print(X_test_df.shape,y_test_df.shape)

In [ ]:



log_reg = LogisticRegression(solver='saga', random_state=0)
log_reg.fit(X_train_df, y_train_df)

svm = SVC(kernel='rbf', random_state=0)
svm.fit(X_train_df, y_train_df)

mlp = MLPClassifier(random_state=0)
mlp.fit(X_train_df, y_train_df)

log_reg_pred = log_reg.predict(X_test_df)
svm_pred = svm.predict(X_test_df)
mlp_pred = mlp.predict(X_test_df)

log_reg_accuracy = accuracy_score(y_test_df, log_reg_pred)
svm_accuracy = accuracy_score(y_test_df, svm_pred)
mlp_accuracy = accuracy_score(y_test_df, mlp_pred)

print("Logistic Regression Accuracy: ", log_reg_accuracy)
print("Support Vector Machine Accuracy: ", svm_accuracy)
print("MLP Classifier Accuracy: ", mlp_accuracy)

print("\nLogistic Regression Classification Report:")
print(classification_report(y_test_df, log_reg_pred))

print("\nSupport Vector Machine Classification Report:")
print(classification_report(y_test_df, svm_pred))

print("\nMLP Classifier Classification Report:")
print(classification_report(y_test_df, mlp_pred))


def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Class 0", "Class 1", "Class 2", "Class 3"], yticklabels=["Class 0", "Class 1", "Class 2", "Class 3"])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

plot_confusion_matrix(y_test_df, log_reg_pred, "Logistic Regression")
plot_confusion_matrix(y_test_df, svm_pred, "Support Vector Machine")
plot_confusion_matrix(y_test_df, mlp_pred, "MLP Classifier")


In [ ]:


# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning)  # General user warnings


# Step 1: Define hyperparameters for tuning

# Logistic Regression and SVM: Tune the 'C' parameter
param_grid_log_reg_svm = {'C': [0.01, 0.1, 1, 10, 20, 50, 100]}

# MLP Classifier: Tune 'hidden_layer_sizes'
param_grid_mlp = {'hidden_layer_sizes': [(128,), (64, 64), (128, 64), (128, 64, 32)]}

# Step 2: Set up GridSearchCV for Logistic Regression and SVM
log_reg = LogisticRegression(solver='saga', random_state=0)
svm = SVC(kernel='rbf', random_state=0)

# Perform GridSearchCV for Logistic Regression and SVM
grid_search_log_reg = GridSearchCV(log_reg, param_grid_log_reg_svm, cv=10, scoring='accuracy')
grid_search_svm = GridSearchCV(svm, param_grid_log_reg_svm, cv=10, scoring='accuracy')

# Reshape y_train and y_test to 1D
y_train_1d = y_train_df.values.ravel()
y_test_1d = y_test_df.values.ravel()

# Fit the models on the training data
grid_search_log_reg.fit(X_train_df, y_train_1d)
grid_search_svm.fit(X_train_df, y_train_1d)

# Step 3: Set up GridSearchCV for MLP
mlp = MLPClassifier(random_state=0)

# Perform GridSearchCV for MLP
grid_search_mlp = GridSearchCV(mlp, param_grid_mlp, cv=5, scoring='accuracy')

# Fit the MLP model on the training data
grid_search_mlp.fit(X_train_df, y_train_1d)

# Step 4: Output the best parameters and the best score for each model

# Logistic Regression Best Parameters and Score
print("Best parameters for Logistic Regression: ", grid_search_log_reg.best_params_)
print("Best cross-validation accuracy for Logistic Regression: ", grid_search_log_reg.best_score_)

# SVM Best Parameters and Score
print("Best parameters for SVM: ", grid_search_svm.best_params_)
print("Best cross-validation accuracy for SVM: ", grid_search_svm.best_score_)

# MLP Best Parameters and Score
print("Best parameters for MLP: ", grid_search_mlp.best_params_)
print("Best cross-validation accuracy for MLP: ", grid_search_mlp.best_score_)

# Step 5: Evaluate the best models on the test set

# Logistic Regression Evaluation
best_log_reg = grid_search_log_reg.best_estimator_
log_reg_pred = best_log_reg.predict(X_test_df)
log_reg_accuracy = accuracy_score(y_test_1d, log_reg_pred)
print("Logistic Regression Test Accuracy: ", log_reg_accuracy)

# SVM Evaluation
best_svm = grid_search_svm.best_estimator_
svm_pred = best_svm.predict(X_test_df)
svm_accuracy = accuracy_score(y_test_1d, svm_pred)
print("SVM Test Accuracy: ", svm_accuracy)

# MLP Evaluation
best_mlp = grid_search_mlp.best_estimator_
mlp_pred = best_mlp.predict(X_test_df)
mlp_accuracy = accuracy_score(y_test_1d, mlp_pred)
print("MLP Test Accuracy: ", mlp_accuracy)


In [ ]:

# Step 1: Retrain models with optimal hyperparameters obtained from GridSearchCV
warnings.filterwarnings('ignore', category=UserWarning)  # General user warnings

# Logistic Regression with optimal C value
best_log_reg = LogisticRegression(solver='saga', C=grid_search_log_reg.best_params_['C'], random_state=0)
best_log_reg.fit(X_train_df, y_train_df)

# SVM with optimal C value
best_svm = SVC(kernel='rbf', C=grid_search_svm.best_params_['C'], random_state=0)
best_svm.fit(X_train_df, y_train_df)

# MLP with optimal hidden_layer_sizes
best_mlp = MLPClassifier(hidden_layer_sizes=grid_search_mlp.best_params_['hidden_layer_sizes'], random_state=0)
best_mlp.fit(X_train_df, y_train_df)

# Step 2: Evaluate models on the test set

# Logistic Regression Evaluation
log_reg_pred = best_log_reg.predict(X_test_df)
log_reg_accuracy = accuracy_score(y_test_df, log_reg_pred)
log_reg_class_report = classification_report(y_test_df, log_reg_pred)

# SVM Evaluation
svm_pred = best_svm.predict(X_test_df)
svm_accuracy = accuracy_score(y_test_df, svm_pred)
svm_class_report = classification_report(y_test_df, svm_pred)

# MLP Evaluation
mlp_pred = best_mlp.predict(X_test_df)
mlp_accuracy = accuracy_score(y_test_df, mlp_pred)
mlp_class_report = classification_report(y_test_df, mlp_pred)

# Step 3: Output the results

print("Logistic Regression - Test Accuracy:", log_reg_accuracy)
print("Logistic Regression - Classification Report:\n", log_reg_class_report)

print("SVM - Test Accuracy:", svm_accuracy)
print("SVM - Classification Report:\n", svm_class_report)

print("MLP - Test Accuracy:", mlp_accuracy)
print("MLP - Classification Report:\n", mlp_class_report)

# Step 4: Plot Confusion Matrices for each model

# Confusion Matrix for Logistic Regression
log_reg_cm = confusion_matrix(y_test_df, log_reg_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(log_reg_cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title("Logistic Regression Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Confusion Matrix for SVM
svm_cm = confusion_matrix(y_test_df, svm_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(svm_cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title("SVM Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Confusion Matrix for MLP
mlp_cm = confusion_matrix(y_test_df, mlp_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(mlp_cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title("MLP Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


Step 6

Performance Comparison Analysis
----
Before Hyperparameter Tuning

Logistic Regression: 75.25%

SVM: 76.90%

MLP: 74.59%

After Hyperparameter Tuning
----
Logistic Regression: 75.25% (unchanged)

SVM: 77.56% (improved by 0.66%)

MLP: 76.90% (improved by 2.31%)

Model Performance Analysis
----
Best Performing Model

SVM consistently performed the best, both before and after tuning

The improvements from hyperparameter tuning were minimal, suggesting the models were already well-configured for the dataset

Class-wise Performance
----
Class 3 shows the highest performance across all models (88-90% precision)

Class 1 shows the most inconsistent performance across models

Classes 0 and 2 show moderate performance (55-75% range)

Possible Reason
----
1. Class Imbalance

    Class 3 has 153 samples

    Class 1 has only 12 samples

    This significant imbalance affects model learning

2. Feature Representation

    Classes 0 and 1 might have overlapping features

    The models struggle to find clear decision boundaries for these classes

3. Data Quality

    The small sample size for Class 1 might not capture all possible variations

    Features might not be discriminative enough for certain classes